In [1]:
import pandas as pd
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split

# Đọc dữ liệu từ file CSV
df = pd.read_csv('transactions.csv')

# Chuyển đổi dữ liệu thành định dạng của Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user_id', 'product_id', 'rating']], reader)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
trainset, testset = train_test_split(data, test_size=0.2)

In [2]:
# Tạo mô hình SVD
model = SVD()

# Huấn luyện mô hình trên tập huấn luyện
model.fit(trainset)

In [3]:
# Tạo dự đoán trên tập kiểm tra
predictions = model.test(testset)

# Đánh giá mô hình
accuracy.rmse(predictions)

# Hiển thị kết quả dự đoán
print("Predictions:")
for uid, iid, true_r, est, _ in predictions[:10]:
    print(f'User {uid:<10} - Product {iid:<10} | Actual: {true_r:<5.2}, Predicted: {est:<5.2f}')

RMSE: 0.1129
Predictions:
User 8611493    - Product 161208247  | Actual: 2.8  , Predicted: 2.81 
User 26069437   - Product 110308041  | Actual: 2.4  , Predicted: 2.45 
User 5870657    - Product 73744766   | Actual: 4.7  , Predicted: 4.60 
User 9678030    - Product 110308041  | Actual: 2.4  , Predicted: 2.44 
User 9910904    - Product 275225043  | Actual: 3.3  , Predicted: 3.40 
User 26806800   - Product 121099479  | Actual: 1.0  , Predicted: 1.09 
User 6920480    - Product 14847948   | Actual: 4.8  , Predicted: 4.67 
User 27060399   - Product 22774705   | Actual: 2.2  , Predicted: 2.19 
User 15938244   - Product 164550314  | Actual: 3.4  , Predicted: 3.38 
User 24908806   - Product 14702607   | Actual: 4.3  , Predicted: 4.20 


In [4]:
# Hàm gợi ý sản phẩm
def get_top_n_recommendations(user_id, n=10):
    # Lấy danh sách tất cả các product_id
    all_product_ids = df['product_id'].unique()
    
    # Dự đoán rating cho tất cả các sản phẩm mà user chưa đánh giá
    user_ratings = [(product_id, model.predict(user_id, product_id).est) for product_id in all_product_ids]
    
    # Sắp xếp theo rating dự đoán
    user_ratings.sort(key=lambda x: x[1], reverse=True)
    
    # Lấy top n sản phẩm
    top_n_recommendations = user_ratings[:n]
    
    return top_n_recommendations

In [5]:
# Ví dụ: Gợi ý 10 sản phẩm cho user_id = 1
recommendations = get_top_n_recommendations(22580465, 10)
print("\nTop Recommendations:")
for product_id, rating in recommendations:
    print(f'Product ID: {product_id:<10}, Predicted Rating: {rating:<5.2f}')


Top Recommendations:
Product ID: 45480982  , Predicted Rating: 5.00 
Product ID: 107709913 , Predicted Rating: 5.00 
Product ID: 46640788  , Predicted Rating: 5.00 
Product ID: 128276418 , Predicted Rating: 5.00 
Product ID: 93710743  , Predicted Rating: 5.00 
Product ID: 114446808 , Predicted Rating: 5.00 
Product ID: 6306239   , Predicted Rating: 5.00 
Product ID: 134025169 , Predicted Rating: 5.00 
Product ID: 207765359 , Predicted Rating: 4.99 
Product ID: 134025507 , Predicted Rating: 4.99 


In [10]:
df['rating']

0        3.5
1        4.0
2        3.4
3        1.0
4        1.7
        ... 
29460    4.0
29461    4.4
29462    3.0
29463    2.6
29464    3.2
Name: rating, Length: 29465, dtype: float64

In [7]:
df.product_id.nunique()

506

In [8]:
# Ví dụ: Gợi ý 10 sản phẩm cho user_id = 1
recommendations = get_top_n_recommendations(16877862, 10)
for product_id, rating in recommendations:
    print(f'Product ID: {product_id}, Predicted Rating: {rating}')

Product ID: 45480982, Predicted Rating: 5
Product ID: 107709913, Predicted Rating: 5
Product ID: 128276418, Predicted Rating: 5
Product ID: 275225050, Predicted Rating: 5
Product ID: 100179757, Predicted Rating: 4.990398419131206
Product ID: 52417141, Predicted Rating: 4.984804237357433
Product ID: 4382605, Predicted Rating: 4.96228210637522
Product ID: 170762091, Predicted Rating: 4.961471721402144
Product ID: 102085482, Predicted Rating: 4.954892344931525
Product ID: 4399961, Predicted Rating: 4.95461839721091
